In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
from google.oauth2 import service_account

path = ''
credentials = service_account.Credentials.from_service_account_file(path + ".json")


In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client(credentials=credentials)

# Construct a reference to the "crypto_bitcoin" dataset
dataset_ref = client.dataset("crypto_bitcoin", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "transactions" table
table_ref = dataset_ref.table("transactions")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

,hash,size,virtual_size,version,lock_time,block_hash,block_number,block_timestamp,block_timestamp_month,input_count,output_count,input_value,output_value,is_coinbase,fee,inputs,outputs
0,a16f3ce4dd5deb92d98ef5cf8afeaf0775ebca408f708b...,275,275,1,0,00000000dc55860c8a29c58d45209318fa9e9dc2c1833a...,181,2009-01-12 06:02:13+00:00,2009-01-01,1,2,4000000000.000000000,4000000000.000000000,False,0E-9,"[{'index': 0, 'spent_transaction_hash': 'f4184...","[{'index': 0, 'script_asm': '04b5abd412d4341b4..."
1,591e91f809d716912ca1d4a9295e70c3e78bab077683f7...,275,275,1,0,0000000054487811fc4ff7a95be738aa5ad9320c394c48...,182,2009-01-12 06:12:16+00:00,2009-01-01,1,2,3000000000.000000000,3000000000.000000000,False,0E-9,"[{'index': 0, 'spent_transaction_hash': 'a16f3...","[{'index': 0, 'script_asm': '0401518fa1d1e1e3e..."
2,12b5633bad1f9c167d523ad1aa1947b2732a865bf5414e...,276,276,1,0,00000000f46e513f038baf6f2d9a95b2a28d8a6c985bcf...,183,2009-01-12 06:34:22+00:00,2009-01-01,1,2,2900000000.000000000,2900000000.000000000,False,0E-9,"[{'index': 0, 'spent_transaction_hash': '591e9...","[{'index': 0, 'script_asm': '04baa9d3665315562..."
3,828ef3b079f9c23829c56fe86e85b4a69d9e06e5b54ea5...,276,276,1,0,00000000fb5b44edc7a1aa105075564a179d65506e2bd2...,248,2009-01-12 20:04:20+00:00,2009-01-01,1,2,2800000000.000000000,2800000000.000000000,False,0E-9,"[{'index': 0, 'spent_transaction_hash': '12b56...","[{'index': 0, 'script_asm': '04bed827d37474bef..."
4,35288d269cee1941eaebb2ea85e32b42cdb2b04284a56d...,277,277,1,0,00000000689051c09ff2cd091cc4c22c10b965eb8db3ad...,545,2009-01-15 05:48:32+00:00,2009-01-01,1,2,2500000000.000000000,2500000000.000000000,False,0E-9,"[{'index': 0, 'spent_transaction_hash': 'd71fd...","[{'index': 0, 'script_asm': '044a656f065871a35..."


In [4]:
def show_amount_of_data_scanned(query):
    # dry_run lets us see how much data the query uses without running it
    dry_run_config = bigquery.QueryJobConfig(dry_run=True)
    query_job = client.query(query, job_config=dry_run_config)
    print('Data processed: {} GB'.format(round(query_job.total_bytes_processed / 10**9, 3)))

In [2]:
import pandas as pd
from time import sleep
import numpy as np

addresses = pd.read_csv(path + 'deanonymised_elliptic.csv')

txns = list(addresses.transaction)
len(txns)


202804

In [7]:
#Ranges for batch processing
ranges_low = [i*15000 for i in range(14)]
ranges_high = [(i+1)*15000 for i in range(14)]
ranges_high[-1] = len(txns) + 1


In [81]:

for i, j in zip(ranges_low, ranges_high):

    query = """
                SELECT `hash`, block_number, block_timestamp, inputs, input_value, is_coinbase, input_count, fee
                FROM `bigquery-public-data.crypto_bitcoin.transactions`
                WHERE `hash` IN UNNEST(""" + str(txns[i:j]) + """)"""
            
    show_amount_of_data_scanned(query)
    data = client.query(query).result().to_dataframe()
    data.to_json(path + f'txn_data/input_{i}-{j}.json')
    print(f'successfully read rows {i} to {j}: data is {data.shape}')
    sleep(np.random.randint(3, 5))
    


Data processed: 1089.059 GB
successfully read rows 0 to 15000: data is (15000, 8)
Data processed: 1089.059 GB
successfully read rows 15000 to 30000: data is (15000, 8)
Data processed: 1089.059 GB
successfully read rows 30000 to 45000: data is (15000, 8)
Data processed: 1089.059 GB
successfully read rows 45000 to 60000: data is (15000, 8)
Data processed: 1089.059 GB
successfully read rows 60000 to 75000: data is (15000, 8)
Data processed: 1089.059 GB
successfully read rows 75000 to 90000: data is (15000, 8)
Data processed: 1089.059 GB
successfully read rows 90000 to 105000: data is (15000, 8)
Data processed: 1089.059 GB
successfully read rows 105000 to 120000: data is (15000, 8)
Data processed: 1089.059 GB
successfully read rows 120000 to 135000: data is (15000, 8)
Data processed: 1089.059 GB
successfully read rows 135000 to 150000: data is (15000, 8)
Data processed: 1089.059 GB
successfully read rows 150000 to 165000: data is (15000, 8)
Data processed: 1089.059 GB
successfully read row

In [117]:
for i, j in zip(ranges_low, ranges_high):

    query = """
                SELECT `hash`, block_hash, block_timestamp, outputs, output_value, output_count
                FROM `bigquery-public-data.crypto_bitcoin.transactions`
                WHERE `hash` IN UNNEST(""" + str(txns[i:j]) + """)"""
            
    show_amount_of_data_scanned(query)
    data = client.query(query).result().to_dataframe()
    data.to_json(path + f'txn_data/output_{i}-{j}.json')
    print(f'successfully read rows {i} to {j}: data is {data.shape}')
    sleep(np.random.randint(3, 5))


Data processed: 544.39 GB
successfully read rows 0 to 15000: data is (15000, 6)
Data processed: 544.39 GB
successfully read rows 15000 to 30000: data is (15000, 6)
Data processed: 544.39 GB
successfully read rows 30000 to 45000: data is (15000, 6)
Data processed: 544.39 GB
successfully read rows 45000 to 60000: data is (15000, 6)
Data processed: 544.39 GB
successfully read rows 60000 to 75000: data is (15000, 6)
Data processed: 544.39 GB
successfully read rows 75000 to 90000: data is (15000, 6)
Data processed: 544.39 GB
successfully read rows 90000 to 105000: data is (15000, 6)
Data processed: 544.39 GB
successfully read rows 105000 to 120000: data is (15000, 6)
Data processed: 544.39 GB
successfully read rows 120000 to 135000: data is (15000, 6)
Data processed: 544.39 GB
successfully read rows 135000 to 150000: data is (15000, 6)
Data processed: 544.39 GB
successfully read rows 150000 to 165000: data is (15000, 6)
Data processed: 544.39 GB
successfully read rows 165000 to 180000: data

In [3]:
txn_files = os.listdir(path + '/txn_data')

In [4]:
# Clean up and merge data
txn_files = os.listdir(path + '/txn_data')

input_data = [pd.read_json(path + 'txn_data/' + i) for i in txn_files if 'input' in i]
output_data = [pd.read_json(path + 'txn_data/' + i) for i in txn_files if 'output' in i]

In [5]:
input_data = pd.concat(input_data)
output_data = pd.concat(output_data)

In [6]:
total_data = input_data.merge(output_data[['hash', 'outputs', 'output_value', 'output_count']], on = 'hash', how = 'left')

In [7]:
def process_in_out(df):
    for j in range(len(total_data)):   
        for i in df.inputs[j]:
            i['hash'] = df.hash[j]
        for i in df.outputs[j]:    
            i['hash'] = df.hash[j]
    inputs = pd.DataFrame([i for sublist in df.inputs for i in sublist])[['index', 'hash', 'spent_transaction_hash', 'addresses', 'value', 'spent_output_index']]
    outputs = pd.DataFrame([i for sublist in df.outputs for i in sublist])[['index', 'hash', 'addresses', 'value']]
    inputs['addresses'] = inputs['addresses'].apply(lambda x: x[0])
    outputs['addresses'] = outputs['addresses'].apply(lambda x: x[0])
    return inputs, outputs


inputs, outputs = process_in_out(total_data)
total_data = total_data.drop(['inputs', 'outputs'], axis = 1)

In [8]:
total_data.head()

,hash,block_number,block_timestamp,input_value,is_coinbase,input_count,fee,output_value,output_count
0,25ed671d23313ab42c5f7adb02241881f103bc96751f0b...,405315,1459546343000,23935560,False,5,184022,23751538,500
1,047743277169fae2946c92bc16098716ff5728f1c0a8d8...,405313,1459545676000,156800462,False,41,368760,156431702,2
2,825d61abde55044a68256b8a5fd8d816ce48771bf63771...,405312,1459545252000,85434,False,7,35000,50434,1
3,be27d58f9d084b20284e29f875d75790b3119e3c49d38c...,405319,1459551951000,22448203,False,80,715080,21733123,2
4,bc5d39de8665550f32c787da6cabe0f3b43a63d0c7716f...,405319,1459551951000,13035575,False,17,250001,12785574,2


In [9]:
inputs.head()

,index,hash,spent_transaction_hash,addresses,value,spent_output_index
0,0,25ed671d23313ab42c5f7adb02241881f103bc96751f0b...,91d48d13b3b14a3ceba77308fc04248cad328c5f7e27e2...,189hYTwr2ufByQiemfoA8Go6h6MYwGeHas,19200.0,29
1,1,25ed671d23313ab42c5f7adb02241881f103bc96751f0b...,6297ac568c8920524d772afeaf2cb34224b0ce56198541...,1D7wdkmi2PQTvPEQxqhtHMM3NoPTDK1dVd,11147876.0,1
2,2,25ed671d23313ab42c5f7adb02241881f103bc96751f0b...,11c4496ccceb1e5ddeb0494d59692e3247b453b9f915bf...,19WuSzgcWrx9u4bNWwiak3x1yK2wWQDvVQ,12000000.0,0
3,3,25ed671d23313ab42c5f7adb02241881f103bc96751f0b...,962813ee8101ccea00104847e8e25a5a6996b7f5fc488d...,1Nf7uW9i6Ao1aRuPH6N2wFUjWo16fZxehp,742000.0,0
4,4,25ed671d23313ab42c5f7adb02241881f103bc96751f0b...,a4f9f06a49b43aa9691d71fdebd8642394c873ae777c89...,1ELKdi1Q4TiehbxxiS1peuZJncUHhbGtso,26484.0,47


In [10]:
outputs.head()

,index,hash,addresses,value
0,0,25ed671d23313ab42c5f7adb02241881f103bc96751f0b...,1NLc8HmV7Tj7PUuK4USvN4GGpzwGcEnvo1,47284.0
1,1,25ed671d23313ab42c5f7adb02241881f103bc96751f0b...,1MGczVD4Go5Pu9iYUhWTqK2HR5qNtw5aqz,47265.0
2,2,25ed671d23313ab42c5f7adb02241881f103bc96751f0b...,1NvA9E79QB8U4aFE7CCnp7Ei6Ei41sKXX8,47256.0
3,3,25ed671d23313ab42c5f7adb02241881f103bc96751f0b...,1CBkF1tvdp7fz3WDLD4QD7eSq36AG5gt65,47250.0
4,4,25ed671d23313ab42c5f7adb02241881f103bc96751f0b...,15bYVHfTNeRwi5Q4c8Qbjzy6QJ7bbXqq4t,47240.0


In [22]:
txn_df = pd.DataFrame(txns, columns = ['hash'])

all(item in list(txn_df.hash) for item in list(total_data.hash))

True

In [11]:
total_data.to_csv(path + 'transactions.csv')
inputs.to_csv(path + 'inputs.csv')
outputs.to_csv(path + 'outputs.csv')